In [2]:
F101 = Integers(101)

In [3]:
F101

Ring of integers modulo 101

In [4]:
#(QL)ai + (QR)bi + (Qoi)ci + (Qmi)aibi + Qco = 0

In [5]:
E = EllipticCurve(F101, [0, 3])

In [6]:
E

Elliptic Curve defined by y^2 = x^3 + 3 over Ring of integers modulo 101

In [7]:
G = E([1,2])

In [8]:
R.<X> = PolynomialRing(F101)

In [9]:
K.<X> = GF(101**2, modulus=X^2+2)

In [10]:
E2 = EllipticCurve(K, [0,3])

In [11]:
E2

Elliptic Curve defined by y^2 = x^3 + 3 over Finite Field in X of size 101^2

In [12]:
G2 = E2([36, 31*X])

In [13]:
N = G.order()

In [14]:
F17 = Integers(N)

In [15]:
#4th roots of power 1
w = vector(F17, [1, 4, -1 , -4])

In [16]:
w

(1, 4, 16, 13)

In [17]:
k1=2; k2=3

In [18]:
k1*w #cosets

(2, 8, 15, 9)

In [19]:
k2*w #cosets

(3, 12, 14, 5)

In [20]:
A = matrix(F17, [[1,1,1,1],[1,4,4**2,4**3],[1,16,16**2,16**3],[1,13,13**2,13**3]])

In [21]:
Ai = A.inverse()

In [22]:
P.<x> = F17[];x=P.0

In [23]:
a = vector(F17, [3,4,5,9])
b = vector(F17, [3,4,5,16])
c = vector(F17, [9,16,25,25])
fa = P(list(Ai*a))
fb = P(list(Ai*b))
fc = P(list(Ai*c))
ql = P(list(Ai*vector(F17, [0,0,0,1])))
qr = P(list(Ai*vector(F17, [0,0,0,1])))
qm = P(list(Ai*vector(F17, [1,1,1,0])))
qc = P(list(Ai*vector(F17, [0,0,0,0])))
qo = P(list(Ai*vector(F17, [-1,-1,-1,-1])))

In [24]:
fa, fb, fc

(3*x^3 + 3*x^2 + 13*x + 1, 13*x^3 + 14*x^2 + 3*x + 7, 4*x^3 + 11*x^2 + 5*x + 6)

In [25]:
ql,qr,qm,qo,qc

(16*x^3 + 4*x^2 + x + 13,
 16*x^3 + 4*x^2 + x + 13,
 x^3 + 13*x^2 + 16*x + 5,
 16,
 0)

In [26]:
Sa = P(list(Ai*vector(F17, [2,8,15,3])));
Sb = P(list(Ai*vector(F17, [1,4,16,12])));
Sc = P(list(Ai*vector(F17, [13,9,5,14])));

In [27]:
Sa(4), Sb, Sc

(8, x^3 + 13*x^2 + 4, 14*x^3 + 3*x^2 + 7*x + 6)

## Running trusted setup cermony

In [28]:
''' The reference string is a list of elliptic curve points
parameterised by a randomly generated secret number s,
which is a part of toxic waste. Any circuit can use similar SRS 
as long as it has enough elements.'''
'''According to the PLONK protocol paper, a circuit with n
 gates requires an SRS with at least n+5 elements '''

'According to the PLONK protocol paper, a circuit with n\n gates requires an SRS with at least n+5 elements '

In [29]:
s = 2 #secret number
srs_1 = vector(F17,[s**p for p in range(0, 7)])
srs_1 = [i*G for i in srs_1]
srs_2 = [G2, 2*G2]
[srs_1, srs_2]

[[(1 : 2 : 1),
  (68 : 74 : 1),
  (65 : 98 : 1),
  (18 : 49 : 1),
  (1 : 99 : 1),
  (68 : 27 : 1),
  (65 : 3 : 1)],
 [(36 : 31*X : 1), (90 : 82*X : 1)]]

# Round1
### * Round1 of the protocol tells us how to encode our assignment vectors a,b,c for later use
### Generate random b1, .., b6 from F17
### a(x) = (b1x + b2).Zh + fa(x)
### b(x) = (b3x + b4).Zh + fb(x)
### c(x) = (b5x + b6).Zh + fc(x)
### Output => [a(s)], [b(s)], [c(s)]

In [30]:
# Zh is a poly that is zero on all elements of our subgroup H
# Zh = x^4 - 1

In [31]:
# Chose random b1,..,b6 in F17 -> (7,4,11,12,16,2)

In [32]:
Zh = x**4-1

In [33]:
a_x = (7*x+4)*Zh+fa
b_x = (11*x+12)*Zh+fb
c_x = (16*x+2)*Zh+fc

In [34]:
a_x, b_x, c_x

(7*x^5 + 4*x^4 + 3*x^3 + 3*x^2 + 6*x + 14,
 11*x^5 + 12*x^4 + 13*x^3 + 14*x^2 + 9*x + 12,
 16*x^5 + 2*x^4 + 4*x^3 + 11*x^2 + 6*x + 4)

In [35]:
# now these polynomials are converted to elliptic curve points 
# which represent these polynomials evaluated at the secret
# number s

In [36]:
a_coeffs = a_x.coefficients()
a_s = 0
for i in range(0,len(a_coeffs)):
    a_s += a_coeffs[i]*srs_1[i]
    
b_coeffs = b_x.coefficients()
b_s = 0
for i in range(0,len(b_coeffs)):
    b_s += b_coeffs[i]*srs_1[i]

c_coeffs = c_x.coefficients()
c_s = 0
for i in range(0,len(c_coeffs)):
    c_s += c_coeffs[i]*srs_1[i]

## here comes the output for round1
[a_s, b_s, c_s]

[(91 : 66 : 1), (26 : 45 : 1), (91 : 35 : 1)]

# Round 2
## Round 2 is about committing to a single polynomial z that encodes all of the copy constraints we discussed earlier.

### 1. Generate random b7, b9 in F17
### 2. Get challenges beta, gamma in F17
### 3. Compute Z(x) as (b7 * x**2 + b8 * x + b9)Zh(x) + acc(x)
### 4. Output [Z(s)]

In [37]:
# let random b7 = 14, b8 = 11, b9 = 7 
# and lets say verifier choses beta = 12, gamma = 13

In [38]:
acc = [1, -1, -1 , -1]
beta = 12
gamma = 13
for i in range(1,4):
    c1 = (a[i-1]+beta*w[i-1]+gamma)
    c2 = (b[i-1]+beta*k1*w[i-1]+gamma)
    c3 = (c[i-1]+beta*k2*w[i-1]+gamma)
    c4 = (a[i-1]+beta*Sa(w[i-1])+gamma)
    c5 = (b[i-1]+beta*Sb(w[i-1])+gamma)
    c6 = (c[i-1]+beta*Sc(w[i-1])+gamma)
    acc[i] = (acc[i-1] * ((c1*c2*c3)/(c4*c5*c6)))
acc

[1, 3, 9, 4]

In [39]:
acc_x = P(list(Ai*vector(F17, acc)))

In [40]:
acc_x

14*x^3 + 5*x^2 + 16*x

In [41]:
b7 = 14; b8 = 11; b9 = 7

In [42]:
Zx = (b7*x**2 + b8*x + b9)*Zh + acc_x

In [43]:
Zx

14*x^6 + 11*x^5 + 7*x^4 + 14*x^3 + 8*x^2 + 5*x + 10

In [44]:
Zx_coeffs = Zx.coefficients()
z_s = 0
for i in range(0,len(Zx_coeffs)):
    z_s += Zx_coeffs[i]*srs_1[i]
z_s

(32 : 59 : 1)

# Round 3
### 1. Compute quotient challenge alpha from F17
### 2. Compute quotient polynomial t(x)
### 3. Split into degree < n+2 polynomials t_lo(x), t_mid(x), t_hi(x) where t(x) = t_lo(x) + x**(n+2) * tmid(x) + x**(2n+4) * t_hi(x)
### 4. output = [ [t_lo(s)] , [t_mid(s)] , [t_hi(s)] ]

In [45]:
alpha = 15

In [46]:
# for computing t(x) we basically need L1 which refers to Lagrange Basis
# Polynomial over roots of unity H. L1 can be found by interpolating the
# vector (1, 0, 0, 0).
L1 = P(list(Ai*vector(F17, [1,0,0,0])));
L1

13*x^3 + 13*x^2 + 13*x + 13

In [47]:
# we will compute tz = t(x)*Zh(x) and then divide by Zh(x) to find t(x)
tz1 = a_x*b_x*qm + a_x*ql + b_x*qr + c_x*qo + qc;
tz2 = ((a_x + beta * x + gamma)*(b_x+beta*k1*x+gamma)*(c_x+beta*k2*x+gamma)*Zx)*alpha
[tz1, tz2]
Zw = Zx(w[1]*x)
tz3_1 = alpha*(a_x + beta * Sa + gamma)
tz3_2 = (b_x + beta * Sb + gamma)
tz3_3 = (c_x + beta * Sc + gamma)
tz3 = -1*(tz3_1 * tz3_2 * tz3_3 * Zw)
tz4 = (Zx-1)*L1*alpha**2
tz = tz1 + tz2 + tz3 + tz4
# print(tz3)
t = P(tz/Zh)
t

11*x^17 + 7*x^16 + 2*x^15 + 16*x^14 + 6*x^13 + 15*x^12 + x^11 + 10*x^10 + 2*x^9 + x^8 + 8*x^7 + 13*x^6 + 13*x^5 + 9*x^3 + 13*x^2 + 16*x + 11

In [48]:
t_list = t.list() #list() includes 0 coeffs too not coefficients()

In [49]:
t_lo = t_list[0:6]
t_mid = t_list[6:12]
t_hi = t_list[12:18]

In [50]:
t_lo_s = 0
for i in range(0,len(t_lo)):
    t_lo_s += t_lo[i]*srs_1[i]
    
t_mid_s = 0
for i in range(0,len(t_mid)):
    t_mid_s += t_mid[i]*srs_1[i]

t_hi_s = 0
for i in range(0,len(t_hi)):
    t_hi_s += t_hi[i]*srs_1[i]

## here comes the output for round1
[t_lo_s, t_mid_s, t_hi_s]

[(12 : 32 : 1), (26 : 45 : 1), (91 : 66 : 1)]

# Round 4
### 1. Compute evaluation challenge zeta from F17
### 2. Compute opening evaluations : ex: a_ = a_x(zeta)
### 3. Compute linearization polynomial r
### 4. Compute linearization evaluation r_
### 5. output => ( a_, b_, c_, sa_, sb_, sc_, zw_, t_, r_)

In [51]:
zeta = 5
a_ = a_x(zeta)
b_ = b_x(zeta)
c_ = c_x(zeta)
sa_ = Sa(zeta)
sb_ = Sb(zeta)
sc_ = Sc(zeta)
t_ = t(zeta)
zw_ = Zx(w[1]*zeta)

In [52]:
r1 = (a_*b_*qm)+(a_*ql)+(b_*qr)+(c_*qo)+qc
r2 = ((a_+beta*zeta+gamma)*(b_+beta*k1*zeta+gamma)*(c_+beta*k2*zeta+gamma)*Zx)*alpha
r3 = ((a_+beta*sa_+gamma)*(b_+beta*sb_+gamma)*beta*zw_*Sc)*alpha
r4 = Zx * L1(zeta) * alpha**2

In [53]:
r = r1 + r2 - r3 + r4
r_ = r(zeta)
r_

15

In [54]:
print(a_, b_, c_, sa_, sb_, sc_, t_, zw_, r_)

15 13 5 1 12 13 1 15 15


# Round 5
### 1. Compute opening challenge vega from F17.
### 2. Compute opening proof polynomial w_zeta(x).
### 3. Compute opening proof polynomial w_zeta_omega(x).
### 4. Output [w_zeta, w_zeta_omega]

In [55]:
vega = 12
v1 = P(t_lo)
v2 = zeta**6*P(t_mid)
v3 = zeta**12*P(t_hi)
v4 = -t_
v5 = vega*(r-r_)+vega**2*(a_x-a_)+vega**3*(b_x-b_)+vega**4*(c_x-c_)+vega**5*(Sa-sa_)+vega**6*(Sb-sb_)
w_z = v1+v2+v3+v4+v5

In [56]:
w_z

5*x^6 + 12*x^5 + 11*x^4 + 8*x^3 + 3*x^2 + 2*x + 5

In [57]:
w_zeta = w_z/(x-zeta)

In [58]:
w_zeta

5*x^5 + 3*x^4 + 9*x^3 + 2*x^2 + 13*x + 16

In [59]:
w_zeta_omega = (Zx-zw_)/(x-zeta*w[1])

In [60]:
w_zeta_omega

14*x^5 + 2*x^4 + 13*x^3 + 2*x^2 + 14*x + 13

In [61]:
w_zeta_coeffs = P(w_zeta).list()
w_zeta_s = 0
for i in range(0, len(w_zeta_coeffs)):
    w_zeta_s += w_zeta_coeffs[i]*srs_1[i]
w_zeta_s

(91 : 35 : 1)

In [62]:
w_zeta_omega_coeffs = P(w_zeta_omega).list()
w_zeta_omega_s = 0
for i in range(0, len(w_zeta_omega_coeffs)):
    w_zeta_omega_s += w_zeta_omega_coeffs[i]*srs_1[i]
w_zeta_omega_s

(65 : 98 : 1)

In [63]:
proof_snark = [a_s, b_s, c_s, z_s, t_lo_s, t_mid_s, t_hi_s, w_zeta_s, w_zeta_omega_s, a_, b_, c_, sa_, sb_, r_, zw_]
proof_snark

[(91 : 66 : 1),
 (26 : 45 : 1),
 (91 : 35 : 1),
 (32 : 59 : 1),
 (12 : 32 : 1),
 (26 : 45 : 1),
 (91 : 66 : 1),
 (91 : 35 : 1),
 (65 : 98 : 1),
 15,
 13,
 5,
 1,
 12,
 15,
 15]

# Verification

## Verifier Preprocessing
### Compute [qm(s)],[ql(s)],[qr(s)],[qo(s)],[qc(s)],[sa(s)],[sb(s)],[sc(s)] using SRS (yep ! common reference string is also known to the verifier)

In [64]:
# qm_s
qm_coeffs = P(qm).list()
qm_s = 0
for i in range(0, len(qm_coeffs)):
    qm_s += qm_coeffs[i]*srs_1[i]

# ql_s
ql_coeffs = P(ql).list()
ql_s = 0
for i in range(0, len(ql_coeffs)):
    ql_s += ql_coeffs[i]*srs_1[i]

# qr_s
qr_coeffs = P(qr).list()
qr_s = 0
for i in range(0, len(qr_coeffs)):
    qr_s += qr_coeffs[i]*srs_1[i]
    
# qo_s
qo_coeffs = P(qo).list()
qo_s = 0
for i in range(0, len(qo_coeffs)):
    qo_s += qo_coeffs[i]*srs_1[i]
    
# qc_s
qc_coeffs = P(qc).list()
qc_s = 0
for i in range(0, len(qc_coeffs)):
    qc_s += qc_coeffs[i]*srs_1[i]
    

# sa_s
sa_coeffs = P(Sa).list()
sa_s = 0
for i in range(0, len(sa_coeffs)):
    sa_s += sa_coeffs[i]*srs_1[i]
    
# sb_s
sb_coeffs = P(Sb).list()
sb_s = 0
for i in range(0, len(sb_coeffs)):
    sb_s += sb_coeffs[i]*srs_1[i]
    
# sc_s
sc_coeffs = P(Sc).list()
sc_s = 0
for i in range(0, len(sc_coeffs)):
    sc_s += sc_coeffs[i]*srs_1[i]

In [65]:
qm_s, ql_s, qr_s, qo_s, qc_s, sa_s, sb_s, sc_s

((12 : 69 : 1),
 (32 : 42 : 1),
 (32 : 42 : 1),
 (1 : 99 : 1),
 0,
 (68 : 74 : 1),
 (65 : 3 : 1),
 (18 : 49 : 1))

In [66]:
upsilon = 4 # randomly chosen by verifier

## Verification Algorithm

## 1. Validate a_s, b_s, c_s, z_s, t_lo_s, t_mid_s, t_hi_s, w_zeta_s, w_zeta_omega_s belong to G1

In [67]:
def validate_part_of_g1(x, y):
    return y**2 == x**3 + 3

In [68]:
verification_algo_1_verified =  validate_part_of_g1(a_s[0],a_s[1]) and validate_part_of_g1(b_s[0],b_s[1]) and validate_part_of_g1(c_s[0],c_s[1]) and validate_part_of_g1(z_s[0],z_s[1]) and validate_part_of_g1(t_lo_s[0],t_lo_s[1]) and validate_part_of_g1(t_mid_s[0],t_mid_s[1]) and validate_part_of_g1(t_hi_s[0],t_hi_s[1]) and validate_part_of_g1(w_zeta_s[0],w_zeta_s[1]) and validate_part_of_g1(w_zeta_omega_s[0],w_zeta_omega_s[1])

verification_algo_1_verified

True

## 2. Validate field elements are a part of F17

In [69]:
verification_algo_2_verified = int(a_)<17 and int(b_)<17 and int(c_)<17 and int(sa_)<17 and int(sb_)<17 and int(r_)<17 and int(zw_)<17

verification_algo_2_verified

True

## 3. Validate pub inputs are a part of F17

## 4. Compute zero polynomial evaluation Zh(zeta) = (zeta**n) - 1

In [70]:
Zh_zeta = F17(zeta**4-1)

In [71]:
Zh_zeta

12

## 5. Compute Lagrange Polynomial
### L1(zeta) = (zeta**n-1)/n * (zeta-1)

In [72]:
L1_zeta = F17((zeta**4 - 1)/(4 * (zeta - 1)))

In [73]:
L1_zeta

5

## 6. Compute public input polynomial evaluation

## 7. Compute quotient polynomial evaluation

In [74]:
t_ =(r_ - ((a_+beta*sa_+gamma)*(b_+beta*sb_+gamma)*(c_+gamma)*zw_)*alpha-L1_zeta*alpha**2)/(Zh(zeta))

In [75]:
t_

1

## 8. Compute first part of batched polynomial commitment [D]

In [76]:
d1 = ZZ(a_*b_*vega)*qm_s + ZZ(a_*vega)*ql_s + ZZ(b_*vega)*qr_s + ZZ(c_*vega)*qo_s + ZZ(vega)*ZZ(qc_s)

In [77]:
d2 = ((a_+beta*zeta+gamma)*(b_+beta*k1*zeta+gamma)*(c_+beta*k2*zeta+gamma)*alpha*vega+L1_zeta*alpha**2*vega+F17(upsilon))*z_s

In [78]:
d2

(26 : 45 : 1)

In [79]:
d3 = -1 * (a_+beta*sa_+gamma)*(b_+beta*sb_+gamma)*alpha*vega*beta*zw_*sc_s

In [80]:
d3

(0 : 1 : 0)

In [81]:
d = d1+d2-d3

In [82]:
d

(0 : 1 : 0)

## 9. Compute full batched polynomial commitment [F]

In [83]:
f = t_lo_s + zeta**6 * t_mid_s + zeta**12 * t_hi_s + d + vega**2 * a_s + vega**3 * b_s + vega**4 * c_s + vega**5 * sa_s + vega**6 * sb_s

In [84]:
f

(68 : 27 : 1)

## 10. Compute group encoded batch Evaluation [E]

In [85]:
e = (t_ + vega*r_ + vega**2*a_ + vega**3*b_ + vega**4*c_ + vega**5 * sa_ + vega**6 * sb_ + upsilon * zw_)*G

In [86]:
e

(1 : 2 : 1)

## 11. Batch Validate all Evaluations (Pairing check)

In [87]:
x1 = w_zeta_s + upsilon *w_zeta_omega_s

In [88]:
x1

(32 : 42 : 1)

In [89]:
x2 = s*G2

In [90]:
x2

(90 : 82*X : 1)

In [91]:
y1 = zeta * w_zeta_s + ZZ(upsilon*zeta*w[1])* w_zeta_omega_s + f - e

In [92]:
y1

(12 : 69 : 1)

In [93]:
y2 = G2

In [94]:
x1_ = E2(x1)
x2_ = E2(x2)
y1_ = E2(y1)
y2_ = E2(y2)

In [95]:
pairing_check_passed = x1_.weil_pairing(x2_, 17) == y1_.weil_pairing(y2_, 17)

In [96]:
if pairing_check_passed:
    print("plonk proof verified")
else:
    print("plonk proof verification failed")

plonk proof verified
